<a href="https://colab.research.google.com/github/cosimo-schiavoni/Massive_Data_Project/blob/main/Downloads_ELEPHAS_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon May 16 03:09:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!wget -q https://www-eu.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
!tar xf spark-3.2.1-bin-hadoop2.7.tgz
!rm spark-3.2.1-bin-hadoop2.7.tgz
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install findspark
!pip install pyspark
!pip install -q kaggle
!pip install elephas
!pip install sparkdl
!pip install tensorframes
!pip install petastorm

     |████████████████████████████████| 281.4 MB 37 kB/s 
     |████████████████████████████████| 198 kB 65.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=c9fa8c8a03cbe360e032fb9b16f93d443cbad03d02a7ba9ea611210b77db212b
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
     |████████████████████████████████| 4.1 MB 44.6 MB/s 
     |████████████████████████████████| 281.3 MB 41 kB/s 
     |████████████████████████████████| 198 kB 62.7 MB/s 
     |████████████████████████████████| 462 kB 57.4 MB/s 
  Created wheel for elephas: filename=elephas-3.1.0-py3-none-any.whl size=26259 sha256=ebbe166af224cfad475a4c1e7e9b0ceeb78a5650e52b95a7bcc0a4819efe2dda
  Stored in directory: /root/.cache/pip/wheels/33/e4/e1/56dda8be927bb0e9971cd7ddf3fc1b17ce78db56268b1f867f
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805911 sha256=05d9

In [3]:
import pyspark
# Spark Session, Pipeline, Functions, and Metrics
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import OneHotEncoder, StringIndexer, StandardScaler, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import rand
from pyspark.mllib.evaluation import MulticlassMetrics

# Keras / Deep Learning
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras import optimizers, regularizers
from tensorflow.keras.optimizers import Adam

# Elephas for Deep Learning on Spark
from elephas.ml_model import ElephasEstimator

In [4]:
from pyspark.sql import SparkSession
from pyspark.ml.image import ImageSchema
from pyspark.sql.functions import lit
from functools import reduce
from google.colab import files
import os
import zipfile
import findspark
from pyspark.sql.functions import col
from petastorm import TransformSpec
from petastorm.spark import SparkDatasetConverter, make_spark_converter
import numpy as np

#from my model
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
import pickle
import shutil
import random
import skimage.io as io
from copy import deepcopy



/usr/local/lib/python3.7/dist-packages/petastorm/spark/spark_dataset_converter.py:28: FutureWarning: pyarrow.LocalFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  from pyarrow import LocalFileSystem


In [5]:
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi

Saving kaggle.json to kaggle.json


In [6]:
os.environ["data_source"] = "./Face_Comics_data"

api = KaggleApi()
api.authenticate()

if not os.path.exists(os.environ["data_source"]):
  os.makedirs(os.environ["data_source"])
  if "comic-faces-paired-synthetic-v2" not in os.listdir():
    if "comic-faces-paired-synthetic-v2.zip" not in os.listdir():
      ! kaggle datasets download -d defileroff/comic-faces-paired-synthetic-v2
      with zipfile.ZipFile("comic-faces-paired-synthetic-v2.zip", 'r') as f:
        f.extractall("comic-faces-paired-synthetic-v2")
    os.remove("comic-faces-paired-synthetic-v2.zip")


!mv "./comic-faces-paired-synthetic-v2/face2comics_v2.0.0_by_Sxela/face2comics_v2.0.0_by_Sxela/comics" "./Face_Comics_data"
!mv "./comic-faces-paired-synthetic-v2/face2comics_v2.0.0_by_Sxela/face2comics_v2.0.0_by_Sxela/faces" "./Face_Comics_data"

!rm -rf comic-faces-paired-synthetic-v2

 99% 2.16G/2.18G [00:13<00:00, 164MB/s]
100% 2.18G/2.18G [00:13<00:00, 172MB/s]


In [7]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"

findspark.init(findspark.find())
#findspark.init(findspark.find())
#spark = SparkSession.builder.master("local").appName("Colab").getOrCreate()
#spark

spark = SparkSession.builder.master("local[*]").appName("Colab").getOrCreate()
conf = SparkConf().setMaster("local[*]").setAppName("Colab")
conf.set("spark.executor.memory", "6G")
conf.set("spark.driver.memory", "2G")
conf.set("spark.executor.cores", "4")
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
conf.set("spark.default.parallelism", "4")
sc =  SparkContext.getOrCreate(conf=conf)
spark

In [8]:
sql_context = SQLContext(sparkContext=spark.sparkContext, sparkSession=spark)

In [9]:
df_comics = spark.read.format("image").load(".//Face_Comics_data//comics").withColumn("label", lit(int(1)))
df_faces = spark.read.format("image").load(".//Face_Comics_data//faces").withColumn("label", lit(int(0)))

df_comics = df_comics.select(col("image.data"),col("label"))
df_faces = df_faces.select(col("image.data"),col("label"))


In [10]:
df_comics.printSchema()

root
 |-- data: binary (nullable = true)
 |-- label: integer (nullable = false)



In [11]:
EPOCHS = 1
BATCH_SIZE = 32
IMG_SIZE = (350, 350)

#Read images and Create training & test DataFrames for transfer learning
#Shuffle Data
df_comics = df_comics.orderBy(rand())
df_faces = df_faces.orderBy(rand())

comics_train, comics_validation, comics_test = df_comics.randomSplit([0.6, 0.2,0.2], seed = 1234)
print(comics_train)
faces_train, faces_validation, faces_test = df_faces.randomSplit([0.6, 0.2,0.2], seed = 1234)
print(faces_train)

#dataframe for training a classification model
train_df = faces_train.unionAll(comics_train)

#dataframe for validating the classification model
val_df = faces_validation.unionAll(comics_validation)

#dataframe for testing the classification model
test_df = faces_test.unionAll(comics_test)


DataFrame[data: binary, label: int]
DataFrame[data: binary, label: int]


In [12]:
#import pandas as pd
#pd_batch0 = train_df.limit(5).toPandas()
#pd_batch0

In [13]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(32, kernel_size=5, strides=2, activation='relu',  input_shape = [350,350,3]))
#Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))
#Convolution
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3,  strides=2, activation = 'relu'))
#Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))
#Second Convolutional Layer
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 2,  strides=2, activation = 'relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides=1, padding='same'))
#Third Convolutional Layer
#cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 2, activation = 'relu'))
#cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides=2))
#Flattening
cnn.add(tf.keras.layers.Flatten())
#Full Connection
cnn.add(tf.keras.layers.Dense(units = 128, activation = 'relu'))
#DROPOUTS ok(0.1)
cnn.add(tf.keras.layers.Dropout(0.1))
#Output Layer
cnn.add(tf.keras.layers.Dense(units=1, activation = 'sigmoid'))
###Training CNN
#Compiling the CNN
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#training the CNN on the train_dataset and validate it on the validation_dataset
#model_fit = cnn.fit(x = train_dataset, validation_data = validation_dataset, epochs = EPOCHS)

cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 173, 173, 32)      2432      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 86, 86, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 42, 42, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 21, 21, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 10, 32)        4128      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 10, 10, 32)       0

In [14]:
from elephas.utils.rdd_utils import to_simple_rdd
from elephas.spark_model import SparkModel
from tensorflow import keras
import tensorflow as tf 

optimizer_conf = tf.keras.optimizers.Adam(lr=0.01)
opt_conf = optimizers.serialize(optimizer_conf)

# Initialize Elephas Spark ML Estimator
estimator = ElephasEstimator()
estimator.setFeaturesCol("data")
estimator.setLabelCol("label")
estimator.set_keras_model_config(cnn.to_json())
estimator.set_categorical_labels(True)
estimator.set_nb_classes(2)
estimator.set_num_workers(6)
estimator.set_epochs(EPOCHS)
estimator.set_batch_size(BATCH_SIZE)
estimator.set_verbosity(1)
estimator.set_validation_split(0.20)
estimator.set_optimizer_config(opt_conf)
estimator.set_mode("synchronous")
estimator.set_loss("categorical_crossentropy")
estimator.set_metrics(['acc'])


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


ElephasEstimator_f634334e81c1

In [15]:
# Fitting a model returns a Transformer
pipeline = Pipeline(stages=[estimator])

In [16]:
#fitted_pipeline = pipeline.fit(train_df)

In [17]:
!pip install datetime

     |████████████████████████████████| 51 kB 645 kB/s 
     |████████████████████████████████| 251 kB 52.7 MB/s 


In [18]:
import datetime

def dl_pipeline_fit_score_results(dl_pipeline=pipeline,
                                  train_data = train_df,
                                  test_data = test_df,
                                  label = 'label'):
  
    
    print(str(datetime.datetime.now().time()) + ': Start Training')
    fit_dl_pipeline = dl_pipeline.fit(train_data)
    pred_train = fit_dl_pipeline.transform(train_data)
    print(str(datetime.datetime.now().time()) + ': Start Testing')
    pred_test = fit_dl_pipeline.transform(test_data)

    pnl_train = pred_train.select(label, "prediction")
    pnl_test = pred_test.select(label, "prediction")

    pred_and_label_train = pnl_train.rdd.map(lambda row: (row[label], row['prediction']))
    pred_and_label_test = pnl_test.rdd.map(lambda row: (row[label], row['prediction']))

    metrics_train = MulticlassMetrics(pred_and_label_train)
    metrics_test = MulticlassMetrics(pred_and_label_test)

    print(str(datetime.datetime.now().time()) + ': Finish')
    print("Training data accuracy: {}".format(round(metrics_train.precision(),4)))
    print("Training Data Confusion Matrix")
    display(pnl_train.crosstab('label','prediction').toPandas())

    print("\nTest data accuracy: {}".format(round(metrics_test.precision(),4)))
    print("Test Data Confusion Matrix")
    display(pnl_test.crosstab('label','prediction').toPandas())
    

In [19]:
dl_pipeline_fit_score_results(dl_pipeline=pipeline,
                                  train_data = train_df,
                                  test_data = test_df,
                                  label = 'label')

03:12:53.201510: Start Training


KeyboardInterrupt: ignored